In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
import scipy as sp
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 8
refRatio = 2

deriv = 'U'
order = 2

In [3]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh_min / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_max = omega.nh_max


In [4]:
def CDStencil(orderIn):
    if (orderIn % 2 == 0):
        order = orderIn
    else:
        order = int(orderIn + 1)
    
    loops = int(order / 2)
    print(loops)
        
    coefs = np.zeros(loops)
    stenc = np.zeros(order + 1)
    terms = np.arange(order + 1)
    rCell = np.asarray([1 / sp.math.factorial(j) for j in terms])
    lCell = rCell + 0
    lCell[1::2] = -lCell[1::2]
    deltaXFunc = lambda k: k ** terms
    tExp = [[] for j in range(loops)]
    for k in range(loops):
        rCellNew = rCell * deltaXFunc(k + 1)
        lCellNew = lCell * deltaXFunc(k + 1)
        tExp[k] = rCellNew - lCellNew


    tExp = np.asarray(tExp).transpose()
#     print('tExp:')
#     print(tExp)
#     print('')
    mat = tExp[1::2, :][1:, :-1]
#     print('mat:')
#     print(mat)
#     print('')

    vec = tExp[1::2, :][1:, -1]
    vec = -vec
#     print('vec:')
#     print(vec)
#     print('')
    coefs[-1] = 1
    coefs[:-1] = LA.inv(mat) @ vec
    stenc[:loops] = -coefs[::-1]
    stenc[loops + 1:] = coefs
    stenc = (-1)**(loops + 1) * stenc
    print(coefs)
    print(stenc)
#     print('coefs:')
#     print(coefs)
#     print('')
#     print('stenc:')
#     print(stenc)
#     print('')
    val = abs((tExp @ coefs)[1])
    stenc = stenc / val
#     print('val:')
#     print(val)
#     print('')
    
    return stenc

In [5]:
stenc = CDStencil(order)
print(stenc)
print('')

3
[45. -9.  1.]
[ -1.   9. -45.   0.  45.  -9.   1.]
[-0.01666667  0.15       -0.75        0.          0.75       -0.15        0.01666667]



In [6]:
def UDStencil(order):
    coefs = np.zeros(order + 1)
    stenc = np.zeros(order + 1)
    if (order == 1):
        faceR = np.asarray([0, 1])
    else:
        if (order == 2):
            faceR = np.asarray([0, 1, 1])
        else:
            if (order == 3):
                faceR = np.asarray([0, 1, 1, 1])
            else:
                if (order == 4):
                    faceR = np.asarray([0, 1, 1, 1, 1])
                else:
                    if (order == 5):
                        faceR = (1. / 60.) * np.asarray([0, 2, -13, 47, 27, -3])
                    else:
                        if (order == 6):
                            faceR = np.asarray([0, 1, 1, 1, 1, 1, 1])
                        else:
                            if (order == 7):
                                faceR = (1. / 420.) * np.asarray([0, -3, 25, -101, 319, 214, -38, 4])
                            else:
                                if (order == 8):
                                    faceR = np.asarray([0, 1, 1, 1, 1, 1, 1, 1, 1])
                                else:
                                    if (order == 9):
                                        faceR = (1. / 2520.) * np.asarray([0, 4, -41, 199, -641, 1879, 1375, -305, 55, -5])
                                    else:
                                        errorMess = 'This program is not designed to handle this order of accuracy for forward- and backward-difference operators.'
    faceL = np.roll(faceR, -1)
    print(faceR)
    print(faceL)
    stenc = faceR - faceL
    
    return stenc

In [7]:
def UDStencil1(order):
    coefs = np.zeros(order + 1)
    stenc = np.zeros(order + 1)
    terms = np.arange(order + 1)
    cell = np.asarray([1 / sp.math.factorial(j) for j in terms])
    deltaXFunc = lambda k: k ** terms
    tExp = [[] for j in range(order + 1)]
    for k in range(order + 1):
        cellNew = cell * deltaXFunc(k)
        tExp[k] = cellNew
    
    tExp = np.asarray(tExp).transpose()
    mat = np.zeros((order, order), float)
    vec = np.zeros(order, float)
    mat[0, :] = tExp[0, :order]
    mat[1:, :] = tExp[2:, :order]
    vec[0] = tExp[0, order]
    vec[1:] = tExp[2:, order]
    vec = -vec
    stenc[order] = 1
    stenc[:order] = LA.inv(mat) @ vec
    val = (tExp @ stenc)[1]
    
#     UNCOMMENT THIS LINE!!!!!
#     stenc = stenc / val



#     print(tExp)
#     print('')
#     print(mat)
#     print('')
#     print(vec)
#     print('')
#     print(val)
#     print('')
#     stenc[:order] = coefs[::-1]
#     stenc[order + 1:] = -coefs
    return stenc

In [8]:
def DDStencil(order):
    stenc = -UDStencil(order)[::-1]
    return stenc

In [9]:
def CentGhost(omega, order, x_0, diff):
    errorLoc = 'ERROR:\nGridTransferTools:\nCentGhost:\n'
    errorMess = ''
    
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    p = np.where(spots > 0)[0][0]
    q = np.where(spots < 0)[0][0]
    
    h_c = max(hs)
    h_f = min(hs)
    
    n_c_m = list(hs).count(h_c)
    n_f_m = list(hs).count(h_f)
    
    ghostCell, n_c, n_f = GTT.GhostCellStencil(order, x_0)
    
    if (n_c > n_c_m):
        errorMess = 'This grid has too few coarse cells for the order of the polynomial interpolation!'
    if (n_f > n_f_m):
        errorMess = 'This grid has too few fine cells for the order of the polynomial interpolation!'
    
    cells = n_c + n_f
    
    # ADD ERROR CHECKER FOR THE NUMBER OF COURSE AND FINE CELLS ON GRID OF GIVEN SIZE!!!
    

    fullStenc = np.zeros(degFreed, float)
    
    if (x_0 > 0):
        for k in range(cells):
            index = (p - n_f + k + 1) % degFreed
            fullStenc[index] = ghostCell[k]
    else:
        if (x_0 < 0):
            for k in range(cells):
                index = (q - n_c + k + 1) % degFreed
                fullStenc[index] = ghostCell[k]
        else:
            errorMess = 'Argument x_0 cannot be 0!'
    
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    
    return fullStenc

In [10]:
ghostCent = CentGhost(omega, order, 0.5, deriv)
print(ghostCent)
print('')
print(omega.xPatches)

0.5
xValsR: [0.03125 0.0625  0.125   0.25    0.5     1.     ]
xValsL: [0. 0. 0. 0. 0. 0.]
[0.03125 0.0625  0.125   0.25    0.5     1.     ]
YOU'RE USING THE UPDATE.
bounds: [-2.  -1.5 -1.  -0.5 -0.   1.   2. ]

[ 0.          0.          0.          0.         -0.0625      0.36607143 -0.88392857  1.11607143  0.48214286 -0.01785714  0.          0.        ]

[[array([0.   , 0.125, 0.25 , 0.375, 0.5  , 0.625, 0.75 , 0.875, 1.   ])], [array([0.    , 0.0625, 0.125 , 0.1875, 0.25  , 0.3125, 0.375 , 0.4375, 0.5   ])]]


In [11]:
def MakeSpaceDeriv(omega, order, diff):
    errorLoc = 'ERROR:\nOperatorTools:\nMakeSpaceDeriv:\n'
    errorMess = ''
    if (diff == 'C' or diff == 'CD'):
        stenc = CDStencil(order)
        if (order % 2 == 0):
            orderStenc = order
        else:
            orderStenc = int(order + 1)
        off = int(orderStenc / 2)
        loBound = -off / 2.
        hiBound = off / 2.
    else:
        orderStenc = order
        if (diff == 'U' or diff == 'UD'):
            stenc = UDStencil(order)
            off = 0
            loBound = 0.
            hiBound = orderStenc / 2.
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = DDStencil(order)
                off = 0
                store = loBound
                loBound = - orderStenc / 2.
                hiBound = 0.
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\' \'CD\', \'UD\', or \'DD\'.'
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    stenc = np.ones(orderStenc + 1)
    
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    # Index before fine-coarse interface
    p = np.where(spots > 0)[0][0]
    # Index before coarse-fine interface
    q = np.where(spots < 0)[0][0]
    
    polyStencSet = [[] for i in range(orderStenc)]
    cellFaces = np.linspace(loBound, hiBound, num = orderStenc + 1)
    zeroLoc = np.where(cellFaces == 0)[0][0]
    cellFaces = np.delete(cellFaces, zeroLoc)
    print(cellFaces)
    print('')
    
    

    for i in range(orderStenc):
        print(i)
        polyStencSet[i] = CentGhost(omega, order, cellFaces[i], diff)
    
    polyStencSet = np.asarray(polyStencSet)
    print(polyStencSet)
    print('')

    IMat = np.eye(degFreed, degFreed)
    
    # YOU'RE GONNA NEED THESE TO RESTRICT FOR HIGHER EVEN ORDERS, TOO.
    
    
    
    polyMatU = IMat + 0
    
    
    mat = np.zeros((degFreed, degFreed), float)
    derivOp = mat + 0
    
    for d in range(orderStenc + 1):
        s = off - d
        print('')
        print('BEGIN ITERATION ' + str(s))
        
        derivMat = mat + 0
        np.fill_diagonal(derivMat, stenc[d])
        derivMat = np.roll(derivMat, s, axis = 0)
        
        polyMat = IMat + 0

        if (s > 0):
            j = off - s
            pAt = p
            pLow = (p - 1) % degFreed
            pHi = (p + 1) % degFreed
            qAt = (q - s + 1) % degFreed #(q + 1) % degFreed
            for i in range (s):
                print('j =', j)
                polyMat[pAt, :] = 0
                polyMat[pAt, pLow:pHi] = 0.5
                polyMat[qAt, :] = polyStencSet[j, :]
                pAt = (pAt - 1) % degFreed
                pLow = (pLow - 2) % degFreed
                pHi = (pHi - 2) % degFreed
                qAt = (qAt + 1) % degFreed
                j = j + 1
                
        if (s < 0):
            j = off # - s - 1
            qAt = (q + 1) % degFreed
            qLow = (q + 1) % degFreed
            qHi = (q + 3) % degFreed
            pAt = (p + 1) % degFreed#p
            for i in range(abs(s)):
                print('j =', j)
                print('pAt is', pAt)
                polyMat[qAt, :] = 0
                polyMat[qAt, qLow:qHi] = 0.5
                polyMat[pAt, :] = polyStencSet[j, :]
                print(polyMat[pAt, :])
                print('')
                qAt = (qAt + 1) % degFreed
                qLow = (qLow + 2) % degFreed
                qHi = (qHi + 2) % degFreed
                pAt = (pAt + 1) % degFreed
                j = j + 1 # - 1
        
        matThis = derivMat @ polyMat
#         print(polyMat)
#         print('')
        print(matThis)
        print('')
        print('END ITERATION ' + str(s))
        print('')
        
        derivOp = derivOp + matThis
    
    hMat = OT.StepMatrix(omega)
    
    derivOp = hMat @ derivOp
        
    return derivOp

In [12]:
A = MakeSpaceDeriv(omega, order, deriv)
print(A)

[ 0.          0.03333333 -0.21666667  0.78333333  0.45       -0.05      ]
[ 0.03333333 -0.21666667  0.78333333  0.45       -0.05        0.        ]
[0.5 1.  1.5 2.  2.5]

0
0.5
xValsR: [0.03125 0.0625  0.125   0.25    0.5     1.     ]
xValsL: [0. 0. 0. 0. 0. 0.]
[0.03125 0.0625  0.125   0.25    0.5     1.     ]
YOU'RE USING THE UPDATE.
bounds: [-2.  -1.5 -1.  -0.5 -0.   1.   2. ]

1
1.0
xValsR: [2. 2. 2. 2. 2. 2.]
xValsL: [0.03125 0.0625  0.125   0.25    0.5     1.     ]
[1. 1. 1. 1. 1. 1.]
YOU'RE USING THE UPDATE.
bounds: [-1.5 -1.  -0.5 -0.   1.   2.   3. ]

2
1.5
xValsR: [22.78125 15.1875  10.125    6.75     4.5      3.     ]
xValsL: [2. 2. 2. 2. 2. 2.]
[7.59375 5.0625  3.375   2.25    1.5     1.     ]
YOU'RE USING THE UPDATE.
bounds: [-1.  -0.5 -0.   1.   2.   3.   4. ]

3
2.0
xValsR: [128.  64.  32.  16.   8.   4.]
xValsL: [22.78125 15.1875  10.125    6.75     4.5      3.     ]
[32. 16.  8.  4.  2.  1.]
YOU'RE USING THE UPDATE.
bounds: [-1.  -0.5 -0.   1.   2.   3.   4. ]

4
2.5
x

SystemExit: ERROR:
GridTransferTools:
CentGhost:
This grid has too few coarse cells for the order of the polynomial interpolation!

/Users/sashacurcic/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
A = MakeSpaceDeriv(omega, order, deriv)
B = OT.SpaceDeriv1(omega, order, deriv)

In [ ]:
print(A)
print('')
print(B)
print('')

In [ ]:
A = np.zeros((4, 4), float)
A[:, 0] = 1

A[0, 1] = - 3. / 2.
A[1, 1] = - 1. / 2.
A[2, 1] = 1. / 2.
A[3, 1] = 3. / 2.

A[0, 2] = 7. / 3.
A[1, 2] = 1. / 3.
A[2, 2] = 1. / 3.
A[3, 2] = 7. / 3.

A[0, 3] = - 15. / 4.
A[1, 3] = - 1. / 4.
A[2, 3] = 1. / 4.
A[3, 3] = 15. / 4.

B = LA.inv(A)

In [ ]:
print(A)
print('')
print(B)
print('')
print(sum(B, axis = 0))